# **Callback functions**

* ## **Early Stopping** (使用方法請查看 overfitting.ipynb)
* ## [**Checkpoint**](#Callbacks-—-Checkpoints)
* ## [**Learning Rate Scheduler**](#Callbacks-—-LearningRateScheduler)
* ## [**TensorBoard**](#Callbacks-—-TensorBoard)

### **匯入所需的函式庫**

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical

from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

### **準備資料**

In [2]:
pkgo = pd.read_csv('./Data/pkgo_nyc_class5.csv')

In [3]:
X_ = pkgo.iloc[:, :-1]
X = X_.values  # 將 dataframe 轉換成 ndarray 的資料型態

In [4]:
# 做 One-hot encoding
y = pkgo['class']
y = to_categorical(y)

In [5]:
# 將資料切分為訓練集和測試集
X_train, X_valid, y_train, y_valid \
    = train_test_split(X, y, test_size=0.1, random_state=40)

In [6]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)

X_train shape: (1652, 200)
y_train shape: (1652, 5)


### **建構模型**

In [7]:
def build_model(X):
    model = keras.Sequential()

    model.add(layers.Dense(128, input_dim=X.shape[1]))
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(256))
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(5))
    model.add(layers.Activation('softmax'))

    return model

* ## **Callbacks — Checkpoints**

In [8]:
model_path = './Data/callbacks_model.h5'  # 模型儲存的位置

# 建立 Checkpoint
checkpoint = \
    callbacks.ModelCheckpoint(model_path,
                              verbose=1,
                              monitor='val_acc',    # 儲存模型的指標
                              save_best_only=True,  # 是否只儲存最好的
                              mode='max')           # 與指標搭配模式

In [9]:
model = build_model(X)

# 編譯模型用以訓練 (設定 optimizer, loss function, metrics, 等等)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])

In [10]:
# 設定訓練參數
batch_size = 16
epochs = 50

In [11]:
# 訓練模型
fitting_history \
    = model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_valid, y_valid),
                verbose=1,
                shuffle=True,
                callbacks=[checkpoint])  # 將 checkpoint 放在 callbacks

Train on 1652 samples, validate on 184 samples
Epoch 1/50
1472/1652 [=========================>....] - ETA: 0s - loss: 1.5517 - acc: 0.4701
Epoch 00001: val_acc improved from -inf to 0.54891, saving model to ./Data/callbacks_model.h5
1652/1652 [==============================] - 1s 534us/sample - loss: 1.5258 - acc: 0.4728 - val_loss: 1.3211 - val_acc: 0.5489
Epoch 2/50
1520/1652 [==========================>...] - ETA: 0s - loss: 1.1984 - acc: 0.5711
Epoch 00002: val_acc improved from 0.54891 to 0.62500, saving model to ./Data/callbacks_model.h5
1652/1652 [==============================] - 0s 127us/sample - loss: 1.2083 - acc: 0.5696 - val_loss: 1.2052 - val_acc: 0.6250
Epoch 3/50
1552/1652 [===========================>..] - ETA: 0s - loss: 1.1347 - acc: 0.5896
Epoch 00003: val_acc did not improve from 0.62500
1652/1652 [==============================] - 0s 115us/sample - loss: 1.1394 - acc: 0.5872 - val_loss: 1.1021 - val_acc: 0.5978
Epoch 4/50
1536/1652 [==========================>...

In [12]:
# 載入儲存的模型
new_model = tf.keras.models.load_model(model_path)

In [13]:
# 檢驗儲存的模型為 validation accuracy 最高的模型
valid_loss, valid_acc = new_model.evaluate(X_valid, y_valid, verbose=0)
print('validation loss:', valid_loss, '\nvalidation accuracy:', valid_acc)

validation loss: 1.0763153563375059 
validation accuracy: 0.7173913


[(Back to Callbacks)](#Callback-functions)

* ## **Callbacks — LearningRateScheduler**

In [14]:
def schedule(epoch):  # 定義 learning rate 根據 epoch 要如何變動
    if epoch < 20:
        return 0.01
    elif epoch < 40:
        return 0.001
    else:
        return 0.0001

In [15]:
# 建立 LearningRateScheduler
lr_schedule = callbacks.LearningRateScheduler(schedule, verbose=1)

In [16]:
model = build_model(X)

# 編譯模型用以訓練 (設定 optimizer, loss function, metrics, 等等)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])

In [17]:
# 設定訓練參數
batch_size = 16
epochs = 50

In [18]:
# 訓練模型
fitting_history \
    = model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_valid, y_valid),
                verbose=1,
                shuffle=True,
                callbacks=[lr_schedule])  # 將 lr_schedule 放在 callbacks

Train on 1652 samples, validate on 184 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/50
1652/1652 [==============================] - 1s 325us/sample - loss: 1.9820 - acc: 0.5006 - val_loss: 1.2211 - val_acc: 0.6033

Epoch 00002: LearningRateScheduler reducing learning rate to 0.01.
Epoch 2/50
1652/1652 [==============================] - 0s 117us/sample - loss: 1.1346 - acc: 0.5805 - val_loss: 1.0308 - val_acc: 0.6522

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01.
Epoch 3/50
1652/1652 [==============================] - 0s 118us/sample - loss: 1.0539 - acc: 0.6029 - val_loss: 0.9700 - val_acc: 0.6576

Epoch 00004: LearningRateScheduler reducing learning rate to 0.01.
Epoch 4/50
1652/1652 [==============================] - 0s 117us/sample - loss: 1.0614 - acc: 0.6077 - val_loss: 0.9987 - val_acc: 0.6359

Epoch 00005: LearningRateScheduler reducing learning rate to 0.01.
Epoch 5/50
1652/1652 [==============================] - 0s 11

[(Back to Callbacks)](#Callback-functions)

* ## **Callbacks — TensorBoard**

In [19]:
tensorboard = callbacks.TensorBoard(log_dir='./Data/logs')

In [20]:
model = build_model(X)

# 編譯模型用以訓練 (設定 optimizer, loss function, metrics, 等等)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])

In [21]:
# 設定訓練參數
batch_size = 16
epochs = 50

In [22]:
# 訓練模型
fitting_history \
    = model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_valid, y_valid),
                verbose=1,
                shuffle=True,
                callbacks=[tensorboard])  # 將 tensorboard 放在 callbacks

Train on 1652 samples, validate on 184 samples
Epoch 1/50
1652/1652 [==============================] - 1s 413us/sample - loss: 1.5599 - acc: 0.4915 - val_loss: 1.3367 - val_acc: 0.5380
Epoch 2/50
1652/1652 [==============================] - 0s 118us/sample - loss: 1.1688 - acc: 0.5757 - val_loss: 1.2510 - val_acc: 0.5870
Epoch 3/50
1652/1652 [==============================] - 0s 116us/sample - loss: 1.0941 - acc: 0.6120 - val_loss: 1.0558 - val_acc: 0.6250
Epoch 4/50
1652/1652 [==============================] - 0s 118us/sample - loss: 1.0390 - acc: 0.6235 - val_loss: 1.0183 - val_acc: 0.6576
Epoch 5/50
1652/1652 [==============================] - 0s 118us/sample - loss: 0.9855 - acc: 0.6344 - val_loss: 1.2220 - val_acc: 0.5489
Epoch 6/50
1652/1652 [==============================] - 0s 116us/sample - loss: 0.9419 - acc: 0.6713 - val_loss: 0.9961 - val_acc: 0.6685
Epoch 7/50
1652/1652 [==============================] - 0s 114us/sample - loss: 0.9023 - acc: 0.6870 - val_loss: 0.9710 - val

## **TensorBoard 視覺化工具**

### 新增(New Launcher) → Other → Tensorboard
![title](./Slides_image/launch_tensorboard.png)

* ### **Scalars**

![title](./Slides_image/Tensorboard_scalars.PNG)

* ### **Graphs**

![title](./Slides_image/Tensorboard_graph.PNG)

[(Back to Callbacks)](#Callback-functions)